In [23]:
from aa_llm_utils.llms.openai import RBIOpenAI

client_id = "dc-pj2kzmc9fsuvmouyfta3ehqqm"
key = {"d": "AdVmOnx2tYXVcUy20b2Gb2TgkjG7zTN-NGlc4HeiIFJ2y70SrczXzzu2vRCF6jNmeMnb_dF4kG3eANNuY-kCDQDxdEEW_Sb9crCBD7ypko8uMSDzYniwhFaDlIfPagQbEe2xnen4jm-jjQN01n8crBjt2nNWeY8VNjwizBdFfyej9o1sHHnSpgrZZ5plO0wEo232IbUSjuAmzyebRmZKFrZriBtoXnkCRGSu9Ps960hCTNwg7Q2B29p54-37nV7I968BGP9TN24xxCfEH-cZSnM86BKSXBiJUdkTMNsrMzQlBtGv2busm8OThgpz1lShHxZnpwkBoLwKOqgrmZ0rsQ", "dp": "WAE8eVJRjU0nDsb99W5kLUbihsM0R2QtNKz0-RJ8ki6vldnhp-msMHl2oYwxxqwYqW4EO0H1TDg2bzUB5LfUnbK1_LOjmNQr5vVm7yH611k3kjkkEWpzNe56zLQqfeLrDUi5A7oboPsK4WKcpVC3O2k-PQNHEsyjkY8XQJl4JF0", "dq": "PGl3DzCwiOaIUgy8lhTjjZdQtBBD93WJ9tiqi4koMM1OXD9Bbd-ZMv-Z0QkYS73FAMYFCGynwJcVVAs9XF4gIfx42aud0anRqxhEfcuwxmsQNg1z2RQPZZmq6fGF6RoJITHKR3VHrgVxeOfijqAif51VDmxtPumaXroVrWREYKE", "e": "AQAB", "kty": "RSA", "n": "qUxpRdukrLznYO5ezWorwW3Ykayc3aBtB9nNKDfBqThzV3Mz49cILbRk-nfOM2Z0faF_oZm1hdFp7MC-xUh6xkqbOrIvHjdTfI5x6a72RRYgkkPYjR5vmYKac-oNNbXOYoVINTrQUpcvoG0n32C4FjerPIQxX7SJ0QXIQDkf9uUavg1hsBVcfHXQPLQhcjWimP3x3bHYfWopPUA9OW81UnjuD6nzGs4yF6XIPL9iid_iBvBAQZzvNlgoQRO0Wfuxr1-vbjgrv_OneA69WEvHe_BzChI1kKlmp1OPltg5ZgfykwvUlGhSTbn0P8vTGNm6nMrX5wBUW0exWIVZnXKqvw", "p": "0xilmyIAaKEM6S0AGQByMfz2XVa8blehos9i8di8yarTr6irXt8mkTV-KeP1m5LuFKN2FuH3QBWc3Nu2QfEO8x6i1y6SBbtrnNfIuTrESivgsyf0ji5nogTERQbLfhLcVzYfViy009GP8SKEtlkOHK0Er_C_MxmQaIQojKC3egc", "q": "zU-k_lm8sGOiOzi_M3l2trSghCDRG6-Ei5WaogHa2vSSeSUWNhIKwsj_h9FlHY00mpBn2E2HovOZjnpB1w1jWn3-Rq8DoLtidh9S-J8SWQxathsAI8nzEL2vBAqeRp-muhrPHEwHelwSCE1BPcrOxaPFvrSXmDmvSnQjJVsXe4k", "qi": "m7PxMw6LTYERSjJ-_1-yjRWP6-DGI3ZZjRZt4r_C3uNbhsOui4cw7Lt_B46gzNxCtnSBAQWpQHg0xduGuKNnf6FzjfRD8YdLJR_WUoSeCB7Du3D0jgHBNntXNDNkDIJtPZflzkgqmrRTA2Vsld7qr0DRqYqrqpGyV0g-qdGaFWE", "kid": "42bae44e-6061-45e1-8572-bd7aae3ef26d"}

llm = RBIOpenAI(key=key, client_id=client_id)

/tmp/ipykernel_30686/3210334984.py:6: DeprecationWarning: OpenAI is deprecated and will be removed in a future release. Please use DynamicString instead.
  llm = RBIOpenAI(key=key, client_id=client_id)


In [77]:
import json

from pydantic import BaseModel, Field, computed_field, TypeAdapter
from itertools import chain
from devtools import pprint
from datetime import datetime

agent_prompt = """
You are a top of the line chat agent trying to provide comprehensive and useful answers to your users.
Always remember that you have tools available and use them if appropriate.
If you are not sure about details regarding a user question you can ask the user for additional input.
"""



class History(BaseModel):
    messages: list[dict] = Field(default_factory=list, description="List of messages in the conversation.")

    def add_system_message(self, content: str):
        self.messages.append({"role": "system", "cotnent": content})
    
    def add_user_message(self, content: str):
        self.messages.append({"role": "user", "content": content})

    def add_tool_message(self, tool_call_id: str, content: str):
        self.messages.append({"role": "tool", "tool_call_id": tool_call_id, "content": content})

history = History()



f_get_delivery_date = {
    "name": "get_delivery_date",
    "description":  (
        "Get the delivery date for a customer's order."
        "Call this whenever you need to know the delivery date of a package, for example when a customer asks 'What is the delivery date of my package'"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The customer's order ID.",
            },
        },
        "required": ["order_id"],
        "additionalProperties": False,
    }
}


f_get_delivery_status = {
    "name": "get_delivery_status",
    "description":  (
        "Get the delivery status for a customer's order."
        "Call this whenever you need to know the delivery status of a package, for example when a customer asks 'What is the delivery status of my package'"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The customer's order ID.",
            },
        },
        "required": ["order_id"],
        "additionalProperties": False,
    }
}


tools= [
    {
        "type": "function", 
        "function": f_get_delivery_date
    },
    {
        "type": "function",
        "function": f_get_delivery_status
    }
]


def get_delivery_status(order_id: str) -> str:
    return "in transport"


def get_delivery_date(order_id: str) -> str:
    return str(datetime.now())

actual_tools = {
    "get_delivery_date": get_delivery_date,
    "get_delivery_status": get_delivery_status
}



class Agent:
    def __init__(self, llm: RBIOpenAI):
        self.llm = llm

    def process_user_message(self, content: str, messages: list[dict]) -> dict:
        history = History(messages=messages)
        history.add_user_message(content)
        choice = self.do_model_call(history)
        return self.handle_choice(choice, history)["message"]

    def handle_choice(self, choice: dict, history) -> dict:
        if "finish_reason" in choice and choice["finish_reason"] == "tool_calls":
            choice = self.handle_tool_calls(choice, history)
            return self.handle_choice(choice, history)
        else:
            return choice
        
    def handle_tool_calls(self, choice: dict, history: History) -> dict:
        message = choice["message"]
        history.messages.append(message)
        tool_calls = message["tool_calls"]
        for tool_call in tool_calls:
            call_id = tool_call["id"]
            function = tool_call["function"]
            name = function["name"]
            arguments = json.loads(function["arguments"])
            print(f"CALLING: {name} with {arguments!r}")
            result = actual_tools[name](*arguments)
            history.add_tool_message(call_id, result)

        return self.do_model_call(history)


    def do_model_call(self, history) -> dict:
        response = self.llm.chat.completions.create(
            messages=history.messages,
            model="gpt-4o",
            tools=tools
        )
        return response.choices[0].dict()
        

def assemble_tool_call_message(stream) -> dict:
    calls = {}
    for choice in stream:
        delta = choice["delta"]
        if delta["tool_calls"] is not None:
            for tool_call in delta["tool_calls"]:
                idx = tool_call["index"]
                if idx not in calls:
                    calls[idx] = {"name": tool_call["function"]["name"], "arguments": tool_call["function"]["arguments"], "id": tool_call["id"]}
                else:
                    calls[idx]["arguments"] += tool_call["function"]["arguments"]
    
    return  {
        "content": None, "role": "assistant", "function_call": None, 
        "tool_calls": [
            {
                "id": c["id"], "function": {"name": c["name"], "arguments": c["arguments"]}, "type": "function"
            } for c in calls.values()
        ]
    }

    

class StreamAgent:
    def __init__(self, llm: RBIOpenAI):
        self.llm = llm

    def process_user_message(self, content: str, messages: list[dict]):
        history = History(messages=messages)
        history.add_user_message(content)
        response = self.do_model_call(history)
        yield from self.handle_stream(response, history)

    def _get_choice(self, chunk) -> dict:
        return chunk.choices[0].dict()


    def handle_tool_calls(self, message: dict, history: History) -> dict:
        history.messages.append(message)
        tool_calls = message["tool_calls"]
        for tool_call in tool_calls:
            call_id = tool_call["id"]
            function = tool_call["function"]
            name = function["name"]
            arguments = json.loads(function["arguments"])
            print(f"CALLING: {name} with {arguments!r}")
            result = actual_tools[name](*arguments)
            history.add_tool_message(call_id, result)

        return self.do_model_call(history)

    
    def handle_stream(self, response, history: History):
        start = self._get_choice(next(response))
        if "tool_calls" in start["delta"] and start["delta"]["tool_calls"] is not None:
            tool_call_message = assemble_tool_call_message(chain([start], map(self._get_choice, response)))
            tool_response = self.handle_tool_calls(tool_call_message, history)
            yield from self.handle_stream(tool_response, history)
        else:
            yield start
            yield from map(self._get_choice, response)

    
    def do_model_call(self, history) -> dict:
        response = self.llm.chat.completions.create(
            messages=history.messages,
            model="gpt-4o",
            tools=tools,
            stream=True
        )
        return response


messages = [
    {"role": "system", "content": agent_prompt},
]

# agent = Agent(llm)
# m = agent.process_user_message("Tell me the delivery date my order", messages)
# print(m)
# messages.append(m)
# m = agent.process_user_message("My order id is 123badger", messages)
# print(m)


In [80]:
agent = StreamAgent(llm)
for r in agent.process_user_message("Tell me the delivery date and status of my order 124badger", messages):
    print(r.get("delta", {}).get("content", ""), end="")


CALLING: get_delivery_date with {'order_id': '124badger'}
CALLING: get_delivery_status with {'order_id': '124badger'}
Your order with ID "124badger" is currently **in transport** and the estimated delivery date is **October 2, 2024**.None